In [29]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn import metrics
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [30]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'tflearn-dnn'

In [31]:
training_features = pd.read_csv('data/training_features.csv')
test_features = pd.read_csv('data/test_features.csv')

In [32]:
X_full = training_features.ix[:,2:]
y_full = training_features.SeriousDlqin2yrs

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y_full, test_size=0.2, stratify=y_full, random_state=42)

In [33]:
y_train_full = y_train_full.astype(np.int)
y_test_full = y_test_full.astype(np.int)

In [34]:
'''rfc = joblib.load('models/rfc-2017-01-07-20-54-40.pkl')'''

feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_full)

model_dir = 'models/%s-%s' % (model_name, timestamp)
m = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[32], n_classes=3, model_dir=model_dir)

m.fit(X_train_full, y_train_full, steps=100000, batch_size=256)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 600, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None), Dimension(79)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=79, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 1117.05, step = 1
INFO:tensorflow:Saving checkpoin

Estimator(params={'enable_centered_bias': True, 'activation_fn': <function relu at 0x7fbc696ca140>, 'weight_column_name': None, 'hidden_units': [32], 'feature_columns': [_RealValuedColumn(column_name='', dimension=79, default_value=None, dtype=tf.float64, normalizer=None)], 'n_classes': 3, 'optimizer': 'Adagrad', 'dropout': None, 'gradient_clip_norm': None, 'num_ps_replicas': 0})

In [35]:
m.evaluate(X_train_full, y_train_full)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=79, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Restored model from models/tflearn-dnn-2017-01-08-17-43-30
INFO:tensorflow:Eval steps [0,inf) for training step 100000.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 100000 step: loss = 0.548651, accuracy = 0.934717


{'accuracy': 0.93471664, 'global_step': 100000, 'loss': 0.54865086}

In [36]:
m.evaluate(X_test_full, y_test_full)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=79, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Restored model from models/tflearn-dnn-2017-01-08-17-43-30
INFO:tensorflow:Eval steps [0,inf) for training step 100000.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 100000 step: loss = 0.533218, accuracy = 0.9346


{'accuracy': 0.9346, 'global_step': 100000, 'loss': 0.53321779}

In [37]:
X_submission = test_features.ix[:,2:]
y_submission = m.predict_proba(X_submission.astype(np.float32))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=79, default_value=None, dtype=tf.float64, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: models/tflearn-dnn-2017-01-08-17-43-30/model.ckpt-100000-?????-of-00001.


In [38]:
submission = pd.DataFrame()
submission['Id'] = test_features.ID.astype(np.int)
submission['Probability'] = y_submission[:,1]
submission.head()

,Id,Probability
0,1,0.000002
1,2,0.000005
2,3,0.000375
3,4,0.052110
4,5,0.000020


In [39]:
flename = 'submissions/submission-%s-%s.csv' % (model_name, timestamp)
submission.to_csv(flename, index=False)
print flename

submissions/submission-tflearn-dnn-2017-01-08-17-43-30.csv
